![image info](https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/images/banner_1.png)

# Proyecto 1 - Predicción de precios de vehículos usados

En este proyecto podrán poner en práctica sus conocimientos sobre modelos predictivos basados en árboles y ensambles, y sobre la disponibilización de modelos. Para su desasrrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 1: Predicción de precios de vehículos usados".

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 4. Sin embargo, es importante que avancen en la semana 3 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 4, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/competitions/miad2024-12-prediccion-precio-vehiculos).

## Datos para la predicción de precios de vehículos usados

En este proyecto se usará el conjunto de datos de Car Listings de Kaggle, donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como: año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

## Ejemplo predicción conjunto de test para envío a Kaggle

En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [8]:
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import sys
import os

# Función para cargar datos
def load_and_preprocess_data():
    # Cargar datos
    dataTraining = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTrain_carListings.zip')
    dataTraining['Price'] = np.log(dataTraining['Price'])
    dataTraining['Mileage'] = np.log(dataTraining['Mileage'])

    # Preparar características
    X = dataTraining[['Year', 'Mileage', 'State', 'Make', 'Model']]
    le_State = LabelEncoder()
    le_Make = LabelEncoder()
    le_Model = LabelEncoder()
    X['State_encoded'] = le_State.fit_transform(X['State'])
    X['Make_encoded'] = le_Make.fit_transform(X['Make'])
    X['Model_encoded'] = le_Model.fit_transform(X['Model'])
    X = X[['Year', 'Mileage', 'State_encoded', 'Make_encoded', 'Model_encoded']]
    y = dataTraining['Price']

    # Dividir datos
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=12)

    # Guardar encoders para uso futuro
    joblib.dump(le_State, 'le_State.pkl')
    joblib.dump(le_Make, 'le_Make.pkl')
    joblib.dump(le_Model, 'le_Model.pkl')

    return X_train, X_test, y_train, y_test

# Función para entrenar el modelo
def train_model(X_train, y_train):
    model = RandomForestRegressor(max_features=4, n_estimators=100, max_depth=19, random_state=1, min_samples_split=17, n_jobs=-1)
    model.fit(X_train, y_train)
    joblib.dump(model, 'model_rf.pkl')  # Guardar el modelo
    return model

# Función para realizar predicciones
def predict_price(features):
    model = joblib.load('model_rf.pkl')
    le_State = joblib.load('le_State.pkl')
    le_Make = joblib.load('le_Make.pkl')
    le_Model = joblib.load('le_Model.pkl')

    # Codificar características
    features_df = pd.DataFrame([features])
    features_df['State'] = le_State.transform(features_df['State'])
    features_df['Make'] = le_Make.transform(features_df['Make'])
    features_df['Model'] = le_Model.transform(features_df['Model'])

    # Realizar predicción
    price_log = model.predict(features_df)
    price = np.exp(price_log[0])  # Convertir de logaritmo
    return price

# Si el script es el principal ejecutado, entrenar el modelo
if __name__ == '__main__':
    X_train, X_test, y_train, y_test = load_and_preprocess_data()
    model = train_model(X_train, y_train)
    print("Model trained and saved successfully.")

Model trained and saved successfully.


In [9]:
from flask import Flask, request, jsonify
from flask_restx import Api, Resource, fields
import joblib
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

api = Api(app, version='1.0', title='Prediccion Precios Vehiculos',
          description='API usa un modelo Random Forest para pedecir precios de vehiculos')

ns = api.namespace('predict', description='Predice precios vehiculos')

parser = api.parser()
parser.add_argument('Year', type=int, required=True, help='Modelo (año) vehiculo', location='args')
parser.add_argument('Mileage', type=float, required=True, help='Cantidad millas rodadas', location='args')
parser.add_argument('State', type=str, required=True, help='Estado donde esta registrado', location='args')
parser.add_argument('Make', type=str, required=True, help='Marca vehiculo', location='args')
parser.add_argument('Model', type=str, required=True, help='modelo vehiculo', location='args')

resource_fields = api.model('Resource', {
    'predicted_price': fields.Float,
})

@ns.route('/')
class VehiclePriceApi(Resource):
    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def get(self):
        args = parser.parse_args()
        # Crear un diccionario con las claves correctas como espera la función predict_price
        features = {
            'State': args['State'],
            'Make': args['Make'],
            'Model': args['Model'],
            'Year': args['Year'],
            'Mileage': args['Mileage']
        }
        predicted_price = predict_price(features)
        return {'predicted_price': predicted_price}

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.3:5000
Press CTRL+C to quit
127.0.0.1 - - [07/May/2024 12:22:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2024 12:22:09] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
127.0.0.1 - - [07/May/2024 12:22:09] "GET /swaggerui/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [07/May/2024 12:22:09] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [07/May/2024 12:22:09] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [07/May/2024 12:22:10] "GET /swagger.json HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2024 12:22:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2024 12:22:45] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
127.0.0.1 - - [07/May/2024 12:22:45] "GET /swaggerui/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [07/May/2024 12:22:45] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [07/May/2024 12:22:45] "GET /swaggerui/swagg

In [2]:
from flask import Flask
from flask_restx import Api, Resource, fields
import joblib
from flask_cors import CORS
from m09_model_deployment import predict_price

# Inicia flask
app = Flask(__name__)
CORS(app)  # Enable CORS for all routes and origins

# API
api = Api(
    app, 
    version='1.0', 
    title='Prediccion Precios Vehiculos',
    description='API usa un modelo Random Forest para pedecir precios de vehiculos'
)

# Create a namespace
ns = api.namespace('predict', description='Predice precios vehiculos')

# Define the parser for incoming request arguments
parser = api.parser()
parser.add_argument(
    'Year', 
    type=int, 
    required=True, 
    help='Modelo (año) vehiculo', 
    location='args')
parser.add_argument(
    'Mileage', 
    type=float, 
    required=True, 
    help='Cantidad millas rodadas', 
    location='args')
parser.add_argument(
    'State', 
    type=str, 
    required=True, 
    help='Estado donde esta registrado', 
    location='args')
parser.add_argument(
    'Make', 
    type=str, 
    required=True, 
    help='Marca vehiculo', 
    location='args')
parser.add_argument(
    'Model', 
    type=str, 
    required=True, 
    help='modelo vehiculo', 
    location='args')

# Define resource fields
resource_fields = api.model('Resource', {
    'predicted_price': fields.Float,
})

# Create a class for the prediction resource
@ns.route('/')
class VehiclePriceApi(Resource):
    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def get(self):
        # Extract query parameters
        args = parser.parse_args()
        # Use the imported function to predict the price
        predicted_price = predict_price(**args)
        # Return the predicted price
        return {'predicted_price': predicted_price}

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.3:5000
Press CTRL+C to quit
127.0.0.1 - - [07/May/2024 11:42:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2024 11:43:00] "GET /swaggerui/droid-sans.css HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2024 11:43:00] "GET /swaggerui/swagger-ui.css HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2024 11:43:00] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2024 11:43:00] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2024 11:43:00] "GET /swagger.json HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2024 11:45:07] "GET /predict/?Year=2017&Mileage=9913&State=FL&Make=Jeep&Model=Wrangler HTTP/1.1" 500 -
Traceback (most recent call last):
  File "c:\Users\edgar\.pyenv\pyenv-win\versions\3.10.5\lib\site-packages\flask\app.py", line 1498, in __call__
    return self.wsgi_app(environ, start_response)
  File "c:\Users\edgar\.pyenv\pyenv-win\versions\3.10.